<a href="https://colab.research.google.com/github/MaximilianoRamirezN/lecturasIIC3633-2021/blob/main/practicos/pyRecLab_SlopeOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Maximiliano Ramírez

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
> Es un algoritmo de filtrado colaborativo ya que los rating inexistentes se calculan a partir de los rating de usuarios que han evaluado tanto items que ha visto el usuario de interés, como el item que se desea predecir.

- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
>Está basando en ambos, ya que toma información de otros usuarios que puntuaron el mismo item, y de otros items puntuados por el mismo usuario.

- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
>Slope One recibe las calificaciones que distintos usuarios han dado a distintos items. Si se desea predecir que calificación dará un usuario i a un item que aún no ha puntuado, debe calcularse el promedio de las diferencias de rating que existen entre un pares de items para obtener la llamada _diferencia_. De esta forma se calculan todas las diferencias entre pares de items, y para predecir la calificación inexistente se pondera la candidad de usuarios que han puntuado un par de items por su respectiva diferencia y se divide por el total de usuarios considerados 

- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
>La ventaja de Slope one es que actualizar las desviaciones es computacionalmente barato ya que sólo se trata de un promedio. Esto hace que el método sea competitivo con respecto a los esquemas basados en memoria.


- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**
Los autores exponen que este método no necesita que el usuario puntue demasiados items, pues siempre es posible pivotear las calificaciones con las de otros usuarios ya existentes para predecir cómo puntuaria el nuevo usuario distintos items.



# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    247      0 --:--:--  0:00:01 --:--:--   248
100 1546k  100 1546k    0     0   801k      0  0:00:01  0:00:01 --:--:-- 10.0M


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    471      0 --:--:-- --:--:-- --:--:--   471
100  385k  100  385k    0     0   376k      0  0:00:01  0:00:01 --:--:--  376k


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    651      0 --:--:-- --:--:-- --:--:--   649
100  230k  100  230k    0     0   297k      0 --:--:-- --:--:-- --:--:--  297k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 234 kB 40.0 MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [7]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [8]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:** debe calcular el promedio de las diferencias de rating entre pares de items, para establecer la preferencia de los usuarios de un item con respecto a otro.

## Paso 6:

Llego la hora de predecir el rating.

In [9]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [10]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [11]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [14]:
#promedio de rating del usuario 457

train_file[train_file['userid'] == 457]['rating'].mean()

4.038461538461538

In [12]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** Los resultados concuerdan con lo esperado del método, naturalmente es muy dificil que la predicción sea de 5 estrellas, pues al tratarse de un promedio de diferencias, se espera que las diferencias convergan a algún valor medio de rating, sin embargo los resultados son bastante satisfactorios.

In [13]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [15]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [16]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** Para muchos usuarios probados las recomendaciones eran bastante similares. Atribuibles a que encotrabamos usuarios vecinos. El usuario de ID:100 presenta gustos con menos similaridad que el usuario 457

In [21]:
# Mediante el método recommend() genereremos una lista top-10 recomendaciones para el usuario ID:100
reclist_slopeone = mySlopeOne.recommend("100", 10)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1653', '1064', '1512', '1523', '1449', '1130', '1122', '868', '814', '48']


In [22]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1653    Entertaining Angels: The Dorothy Day Story (1996)
1064                                     Crossfire (1947)
1512               World of Apu, The (Apur Sansar) (1959)
1523                         Good Man in Africa, A (1994)
1449                               Pather Panchali (1955)
1130                                Jupiter's Wife (1994)
1122                       They Made Me a Criminal (1939)
868                               Hearts and Minds (1996)
814                         Great Day in Harlem, A (1994)
48                                     Hoop Dreams (1994)
Name: title, dtype: object

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:** Las recomendaciones principalmente no tienen titulos en común como era de esperar, ya que se compara un algoritmo que da recomendaciones personalizadas versus uno que lo hace de forma más personalizada y colaborativa entre usuarios. sin embargo, podemos encontrar coincidencia en los géneros y tramas de peliculas, por ejemplo most popular entrega como recomendación *The Godfather*, y Slope one entrega *Some Mother's Son*, que pueden ser peliculas con similitudes.

## Most-Popular

In [23]:
# Definicion de objeto "most popular"
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)

# en teoria no es "entrenar" , objeto "most_popular" solo registra los items más populares en el set de entrenamiento  
most_popular.train()

In [30]:
# Calcular las recomendaciones para un usuario en particular (id =44)
user_id = 44

ranking = [int(r) for r in most_popular.recommend(str(user_id), 10, includeRated=False)] #Se genera el top 10 recomendaciones
print('Recommendation for user {}: {}'.format(user_id, ranking))
print('\n ################ \n ')

info_file.loc[ranking]['title']

Recommendation for user 44: [50, 100, 181, 286, 1, 121, 300, 127, 7, 98]

 ################ 
 


movieid
50                     Star Wars (1977)
100                        Fargo (1996)
181           Return of the Jedi (1983)
286         English Patient, The (1996)
1                      Toy Story (1995)
121       Independence Day (ID4) (1996)
300                Air Force One (1997)
127               Godfather, The (1972)
7                 Twelve Monkeys (1995)
98     Silence of the Lambs, The (1991)
Name: title, dtype: object

## Slope One

In [31]:
# Mediante el método recommend() genereremos una lista top-10 recomendaciones para el usuario ID:44
reclist_slopeone = mySlopeOne.recommend("44", 10)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)
print('\n ################ \n ')

# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items según ID: ['1656', '1064', '1643', '1642', '1625', '1599', '1512', '1536', '1500', '1467']

 ################ 
 
Lista de items por nombre:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
1536                      Aiqing wansui (1994)
1500                 Santa with Muscles (1996)
1467      Saint of Fort Washington, The (1993)
Name: title, dtype: object